In [ ]:
from sqlalchemy import create_engine
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv

def create_connection():

    load_dotenv()
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')

    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    return conn

# устанавливаем соединение с базой
conn = create_connection()

In [ ]:
data = pd.read_sql('select * from flats_buildings_clean', conn)
data.head()

In [ ]:
data.drop(columns=['id'], inplace=True)
data.head()

In [ ]:
# Разделение данных
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import CatBoostEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from catboost import CatBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = data[(data['price'] > 1_000_000) & (data['price'] < 50_000_000)]

x = data.drop(columns='price')
y = data['price']

# Цена - непрерывный числовой показатель, нужно произвести бинирование - 
# разбиение цен на несколько групп с примерно одинаковым количеством объектов
# в нашем случае выделяем 10 групп и предотвращаем их повторение - duplicates='drop'
price_bins = pd.qcut(y, q=10)

x_tr, x_val, y_tr, y_val = train_test_split(
    x, y,
    test_size=0.2,
    stratify=price_bins,
    random_state=42
)

original_y_val = data.loc[x_val.index, 'price']

# Тренировочная выборка
num_features_tr = x_tr.select_dtypes(include=['float', 'int']).drop(columns=['building_type_int'])
cat_features_tr = x_tr[['building_type_int']]
binary_cat_features_tr = x_tr.select_dtypes(include='bool')

# Валидационная выборка
num_features_val = x_val.select_dtypes(include=['float', 'int']).drop(columns=['building_type_int'])
cat_features_val = x_val[['building_type_int']]
binary_cat_features_val = x_val.select_dtypes(include='bool')

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Обертка для CatBoostEncoder
class CatBoostEncoderWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = CatBoostEncoder()
        self.feature_names = None

    def fit(self, X, y=None):
        self.encoder.fit(X, y)
        self.feature_names = X.columns.tolist()
        return self

    def transform(self, X):
        return self.encoder.transform(X)

    def get_feature_names_out(self, input_features=None):
        return self.feature_names

# определите список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', OneHotEncoder(drop='if_binary', sparse_output=False), binary_cols),
        ('non_binary', CatBoostEncoderWrapper(), non_binary_cat_cols),
        ('scaler', StandardScaler(), num_cols)
    ],
    verbose_feature_names_out=False
)

# трансформируйте исходные данные data с помощью созданного preprocessor
x_tr_transformed = preprocessor.fit_transform(x_tr, y_tr)
x_val_transformed = preprocessor.transform(x_val)


model = CatBoostRegressor(
    verbose=100,
    random_seed=42
)

# Пайплайн
pipeline = Pipeline(
    [
        ('preprocessing', preprocessor),
        ('model', model)
    ]
)

# Обучение пайплайна
pipeline.fit(x_tr, y_tr)

# Предсказания
y_pred = pipeline.predict(x_val)


print('MAE:', mean_absolute_error(y_val, y_pred))
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False)) 
print('R²:', r2_score(y_val, y_pred))

# MAE: 38827.08915154271
# RMSE: 97818.44800851807
# R²: 0.9998350659848153


In [ ]:
# Кросс-валидация
from sklearn.model_selection import cross_val_score

neg_mae_scores = cross_val_score(
    pipeline, x, y,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

mae_scores = -neg_mae_scores
print('MAE на кросс-валидации (по фолдам):', mae_scores)
print('Средний MAE:', mae_scores.mean())
print('Стандартное отклонение:', mae_scores.std())

# MAE на кросс-валидации (по фолдам): [34880.3487744  33100.63965542 38203.81256636 42439.58160784
#  39860.29543549]
# Средний MAE: 37696.93560790356
# Стандартное отклонение: 3361.215225146131

In [ ]:
# Гистограмма ошибок
import matplotlib.pyplot as plt

errors = y_pred - original_y_val

plt.figure(figsize=(10, 5))
plt.hist(errors, bins=100, edgecolor='k')
plt.title('Распределение ошибок (y_pred - y_true)')
plt.xlabel('Ошибка')
plt.ylabel('Частота')
plt.grid(True)
plt.show()

In [ ]:
# MAE по диапазонам цен

df_val = x_val.copy()
df_val['true_price'] = original_y_val
df_val['pred_price'] = y_pred
df_val['error'] = df_val['pred_price'] - df_val['true_price']
df_val['abs_error'] = df_val['error'].abs()

# Разделим на 5 бинов по цене
df_val['price_bin'] = pd.qcut(df_val['true_price'], q=5)

# Средняя абсолютная ошибка по каждому бинe
bin_mae = df_val.groupby('price_bin')['abs_error'].mean()
print('MAE по диапазонам цен:')
print(bin_mae)

# MAE по диапазонам цен:
# price_bin
# (1489999.999, 8300000.0]     23340.602050
# (8300000.0, 10340000.0]      14716.674654
# (10340000.0, 12800000.0]     14667.359793
# (12800000.0, 17300000.0]     22178.545203
# (17300000.0, 49999000.0]    119225.281567
# Name: abs_error, dtype: float64

In [ ]:
import joblib
pipeline.fit(x_tr, y_tr)

with open('fitted_model.pkl', 'wb') as fd:
    joblib.dump(pipeline, fd)